# Notebook - ATAC-seq and scATAC-seq (in-class)

We thank the ENCODE consortium as well as Satija and Stuart team for their software and package tutorials, from which much of the material below is adapted. We thank the Epigenomics Workshop 2024 for posting educational materials online, which were also adapted in creating this notebook. 

### [[Important!!]] Instruction for running this notebook on Cocalc:
Please follow the instructions below to configure your own environment to run this notebook. 

1. In the side bar, Find the "Settings" button, and click the triangle butten next to it.
2. Under the "Control" drop-down manu, change the Software environment to "2024-02-07", then save changes. 
3. Open a Linux Terminal
4. Type “R” to start an R session
5. Type the following commands sequentially. Always **skip** any software updates
 - require(devtools)
 - install_version(“SeuratObject”, version=“5.0.1”)
 - install_version(“Signac”, version=“1.13.0”)
 - install.packages(“irlba”)
 - devtools::install_github(‘immunogenomics/presto’)

## Objectives of this notebook

The objective of this notebook is to get students familiarize with data analysis on bulk and single-cell ATAC-seq data. This notebook will guide students through the steps of quality control, clustering, exploratory data analysis, and differential accessibility analysis. By the end of this process, we aim to uncover insights into the chromatin accessibility landscape across different conditions or cell types, and be able to process both public and primary datasets.

## Setup -- ATAC-seq

The ENCODE consortium provides a very useful tool for analyzing the bulk ATAC-seq data. Please take a look at their workflow, and answer Q1. https://www.encodeproject.org/pipelines/ENCPL787FUN/ (we recommend opening the website using safari as you may encounter display issues with Chrome).

Q1. What is the input and output of the pipeline? 

To run the ENCODE pipeline, a `.json` input file is required. All the experimental summary information, `.fastq` file location, and R1 and R2 read file locations should be provided in this file in order for the pipeline to recognize. Here, we will not actually run the whole pipeline, but we will edit the following `.json` file to mimic what we will need to do in running the pipeline. 

**Q2.** Instruction for editing the input `.json` file can be found here: https://github.com/ENCODE-DCC/atac-seq-pipeline/blob/master/docs/input_short.md  Please read the instruction and edit the `.json` file in the text block below, with the information provided here:

1. The data are collected from mouse liver
2. You don't know the adapter, so you want the algorithm to automatically detect the adapter

In your analysis, don't forget to double check the input `.fastq` files. 

In [ ]:
{
    "atac.title" : "Example (paired end)",
    "atac.description" : "This is a template input JSON for paired ended sample.",

    "atac.pipeline_type" : "atac",
    "atac.align_only" : false,
    "atac.true_rep_only" : false,

    "atac.genome_tsv" : "/path_to_genome_data/hg38/hg38.tsv",

    "atac.paired_end" : true,

    "atac.fastqs_rep1_R1" : [ "rep1_R1_L1.fastq.gz", "rep1_R1_L2.fastq.gz", "rep1_R1_L3.fastq.gz" ],
    "atac.fastqs_rep1_R2" : [ "rep1_R2_L1.fastq.gz", "rep1_R2_L2.fastq.gz", "rep1_R2_L3.fastq.gz" ],
    "atac.fastqs_rep2_R1" : [ "rep2_R1_L1.fastq.gz", "rep2_R1_L2.fastq.gz" ],
    "atac.fastqs_rep2_R2" : [ "rep2_R2_L1.fastq.gz", "rep2_R2_L2.fastq.gz" ],

    "atac.auto_detect_adapter" : false,
    "atac.adapter" : "AATTCCGG",
    "atac.adapters_rep1_R1" : [ "AATTCCGG", "AATTCCGG", "AATTCCGG" ],
    "atac.adapters_rep1_R2" : [ "AATTCCGG", "AATTCCGG" ],
    "atac.adapters_rep2_R1" : [ "AATTCCGG", "AATTCCGG", "AATTCCGG" ],
    "atac.adapters_rep2_R2" : [ "AATTCCGG", "AATTCCGG" ],

    "atac.multimapping" : 4
}

## Setup -- scATAC-seq

Just like for scRNA-seq, many tools have been developed for the analysis of scATAC-seq data. These packages include `Signac`, `ArchR`, `snapATAC (v1)` in **R** and `snapATAC (v2)`, `EpiScanpy` in **Python**. Since we have used `Seurat` for analyzing scRNA-seq data, we will use `Signac` (developed by the same lab) for the scATAC-seq data analysis. 

Just like `Seurat`, `Signac` have many useful vigenettes as well: https://stuartlab.org/signac/ 

Here, we will be analyzing a single-cell ATAC-seq dataset collected from Human PBMC by 10x Genomics. 

### Loading packages

In [ ]:
library(Seurat)
library(Signac)
packageVersion("Seurat")
library(tidyverse)
# library(EnsDb.Hsapiens.v86)
library(ggplot2)
library(patchwork)
library("presto")

## Loading scATAC-seq data
From the `CellRanger-ATAC` pipeline, four files will be used for constructing the `Signac` object, they are:

- A count file. The rows are regions (peaks) and the colums are cells. Each entry i,j is the number of reads mapping to region i in cell j. In this assignment, our file is `atac_pbmc_500_nextgem_filtered_peak_bc_matrix.h5`.


- A meta data file, with some overall statistics for each cell. In this assignment, our file is `atac_pbmc_500_nextgem_singlecell.csv`.


- A fragment file, with information on all sequenced fragments (where it maps to the genome, which cell barcode is associated and how many PCR duplicates were found). In this assignment, our file is `atac_pbmc_500_nextgem_fragments_sub.tsv.gz` 


- An index file connected to the fragment file. This is like an index file for a bam file, to make it possible to quickly find fragments for a certain genomic region, without having to search the entire file. In this assignment, our file called `atac_pbmc_500_nextgem_fragments_sub.tsv.gz.tbi`. (Note that you won't specify this file, but is required. We pre-created this file for you using `tabix` in UNIX.)

### Create Seurat object

Using the above files, edit the code below to specify the file names in the places indicated

In [ ]:
counts <- Read10X_h5(filename = "") ##Specify the count file here

metadata <- read.csv(
  file = "", ##Specify the meta data file here
  header = TRUE,
  row.names = 1
)

chrom_assay <- CreateChromatinAssay(
  counts = counts,
  sep = c(":", "-"),
  fragments = "", ##Specify the fragment file here
  min.cells = 10,
  min.features = 200
)

pbmc <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks",
  meta.data = metadata
)

We can also add gene annotations. This will allow downstream functions to pull the gene annotation information directly from the object.

In [ ]:
# # the coCalc system has trouble installing the lastest annotation, So let us skip this step in class 

# # extract gene annotations from EnsDb
# annotations <- GetGRangesFromEnsDb(ensdb = ensdb_v86)

# # change to UCSC style since the data was mapped to hg38
# seqlevels(annotations) <- paste0('chr', seqlevels(annotations))
# genome(annotations) <- "hg38"

# # add the gene information to the object
# Annotation(pbmc) <- annotations

### Computing QC metrics 
We have introduced these common QC metrics in our prelab notebook. Please refer to that notebook or the Signac website (https://stuartlab.org/signac/articles/pbmc_vignette) for more information. 

In [ ]:
# compute nucleosome signal score per cell
pbmc <- NucleosomeSignal(object = pbmc)

# # compute TSS enrichment score per cell
# pbmc <- TSSEnrichment(object = pbmc)

# add fraction of reads in peaks
pbmc$pct_reads_in_peaks <- pbmc$peak_region_fragments / pbmc$passed_filters * 100

# # add blacklist ratio
# pbmc$blacklist_ratio <- FractionCountsInRegion(
#   object = pbmc, 
#   assay = 'peaks',
#   regions = blacklist_hg38_unified
# )

The enrichment of Tn5 integration events at transcriptional start sites (TSSs) can also be an important quality control metric to assess the targeting of Tn5 in ATAC-seq experiments. The ENCODE consortium defined a TSS enrichment score as the number of Tn5 integration site around the TSS normalized to the number of Tn5 integration sites in flanking regions. See the ENCODE documentation for more information about the TSS enrichment score (https://www.encodeproject.org/data-standards/terms/). We can calculate the TSS enrichment score for each cell using the TSSEnrichment() function in Signac.

In [ ]:
pbmc$nucleosome_group <- ifelse(pbmc$nucleosome_signal > 4, 'NS > 4', 'NS < 4')
FragmentHistogram(object = pbmc, group.by = 'nucleosome_group')

In [ ]:
VlnPlot(
  object = pbmc,
  features = c('nCount_peaks', 'nucleosome_signal', 'pct_reads_in_peaks'),
  pt.size = 0.1,
  ncol = 3
)

### Filtering based on QC metric

Here, we will set some initial QC filters, to focus on a subset of peaks with the following properties:

- read count in peaks greater than 9,000 (i.e., insist on a minimum read depth in a peak)
- read count in peak less than 100,000 (i.e., if a peak has too much depth, exclude)
- minimum fraction of read coverage in peaks (i.e., 40%)

In [ ]:
pbmc <- subset(
  x = pbmc,
  subset = nCount_peaks > 9000 &
    nCount_peaks < 100000 &
    pct_reads_in_peaks > 40 
)
pbmc

### Normalization and linear dimensional reduction

Next, as described in the prelab, we will normalize the subset of peaks we selected above using Term Frequency-Inverse Document Frequency (TF-IDF), and then perform dimensionality reduction for interpretive purposes using singular value decomposition (SVD).

In [ ]:
pbmc <- RunTFIDF(pbmc)
pbmc <- FindTopFeatures(pbmc, min.cutoff = 'q0')
pbmc <- RunSVD(pbmc)

The first LSI component often captures sequencing depth (technical variation) rather than biological variation. If this is the case, the component should be removed from downstream analysis. We can assess the correlation between each LSI component and sequencing depth using the DepthCor() function:

In [ ]:
DepthCor(pbmc)

Here we see there is a very strong correlation between the first LSI component and the total number of counts for the cell, so we will perform downstream steps without this component.

### Non-linear dimension reduction and clustering
Now that the cells are embedded in a low-dimensional space, we can use methods commonly applied for the analysis of scRNA-seq data to perform graph-based clustering, and non-linear dimension reduction for visualization. The functions `RunUMAP()`, `FindNeighbors()`, and `FindClusters()` all come from the Seurat package.

In [ ]:
pbmc <- RunUMAP(object = pbmc, reduction = 'lsi', dims = 2:30)
pbmc <- FindNeighbors(object = pbmc, reduction = 'lsi', dims = 2:30)
pbmc <- FindClusters(object = pbmc, verbose = FALSE, algorithm = 3)
DimPlot(object = pbmc, label = TRUE) + NoLegend()

**Note**: In this notebook, we will skip the integration with scRNA-seq part. Based on various benchmark efforts, the integration between unmatched scRNA-seq and scATAC-seq can be very challenging. We recommend running this type of analysis with caution. 

### Find differentially accessible peaks between clusters
To find differentially accessible regions between clusters of cells, we can perform a differential accessibility (DA) test. A simple approach is to perform a Wilcoxon rank sum test, and the presto package has implemented an extremely fast Wilcoxon test that can be run on a Seurat object.

In [ ]:
DefaultAssay(pbmc) <- 'peaks'

da_peaks <- FindMarkers(
  object = pbmc,
  ident.1 = c("0"), 
  ident.2 = c("1"), 
  test.use = 'wilcox',
  min.pct = 0.1
)

head(da_peaks)

In [ ]:
plot1 <- VlnPlot(
  object = pbmc,
  features = rownames(da_peaks)[1],
  pt.size = 0.1,
  idents = c("0","1")
)
plot2 <- FeaturePlot(
  object = pbmc,
  features = rownames(da_peaks)[1],
  pt.size = 0.1
)

plot1 | plot2